In [7]:
import nltk
nltk.download('wordnet')

from nltk.corpus import wordnet as wn

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
[nltk_data] Downloading package wordnet to /home/njh/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [8]:
def get_hyponyms(synset):
    hyponyms = set()
    for hyponym in synset.hyponyms():
        hyponyms |= set(get_hyponyms(hyponym))
    return hyponyms | set(synset.hyponyms())

In [9]:
female_words = []
for s in get_hyponyms(wn.synset('woman.n.01')):
    female_words.extend(s.lemma_names())
for s in get_hyponyms(wn.synset('girl.n.01')):
    female_words.extend(s.lemma_names())
for i in range(len(female_words)):
    female_words[i] = female_words[i].replace("-", " ")
    female_words[i] = female_words[i].replace("_", " ")
print("- There are {} words in female words dataset.".format(len(female_words)))
print("  Samples: ", ", ".join(female_words[-10:]), ", ...")
    
male_words = []
for s in get_hyponyms(wn.synset('man.n.01')):
    male_words.extend(s.lemma_names())
for s in get_hyponyms(wn.synset('boy.n.01')):
    male_words.extend(s.lemma_names())
for i in range(len(male_words)):
    male_words[i] = male_words[i].replace("-", " ")
    male_words[i] = male_words[i].replace("_", " ")
print("- There are {} words in male words dataset.".format(len(male_words)))
print("  Samples: ", ", ".join(male_words[-10:]), ", ...")

- There are 298 words in female words dataset.
  Samples:  party girl, ring girl, belle, mill girl, colleen, sex kitten, sexpot, sex bomb, peri, flapper , ...
- There are 152 words in male words dataset.
  Samples:  bat boy, Eagle Scout, Boy Scout, schoolboy, ball boy, farm boy, Sea Scout, catamite, shop boy, Scout , ...


In [10]:
import json
with open('FEMALE_words.txt', 'w') as f:
    f.write(json.dumps(female_words))
with open('MALE_words.txt', 'w') as f:
    f.write(json.dumps(male_words))

In [11]:
import numpy as np
import pandas as pd

from sentence_transformers import SentenceTransformer

In [12]:
model = SentenceTransformer('bert-large-nli-stsb-mean-tokens')
female_embeddings = model.encode(female_words)
male_embeddings = model.encode(male_words)

In [14]:
n_users = 7700

female_userIDs = []
r_female_words = []
r_female_embeddings = []

for i, w in enumerate(female_words):
    for userId in range(n_users):
        female_userIDs.append(userId)
        r_female_words.append(w)
        r_female_embeddings.append(female_embeddings[i])

male_userIDs = []
r_male_words = []
r_male_embeddings = []

for i, w in enumerate(male_words):
    for userId in range(n_users):
        male_userIDs.append(userId)
        r_male_words.append(w)
        r_male_embeddings.append(male_embeddings[i])

female_df = pd.DataFrame({'userId': female_userIDs, 'word': r_female_words, 'embedding': r_female_embeddings, 'CAV_label': np.zeros(len(female_userIDs)).tolist()})
male_df = pd.DataFrame({'userId': male_userIDs, 'word': r_male_words, 'embedding': r_male_embeddings, 'CAV_label': np.ones(len(male_userIDs)).tolist()})

print("female words\n", female_df.head())
print("male words\n", male_df.head())

cav_df = pd.concat([female_df, male_df])
print("concatenated CAV dataframe\n", cav_df.head())

female words
    userId         word                                          embedding  \
0       0  white woman  [-0.680018, 0.57383853, -0.50681406, -0.333249...   
1       1  white woman  [-0.680018, 0.57383853, -0.50681406, -0.333249...   
2       2  white woman  [-0.680018, 0.57383853, -0.50681406, -0.333249...   
3       3  white woman  [-0.680018, 0.57383853, -0.50681406, -0.333249...   
4       4  white woman  [-0.680018, 0.57383853, -0.50681406, -0.333249...   

   CAV_label  
0        0.0  
1        0.0  
2        0.0  
3        0.0  
4        0.0  
male words
    userId             word                                          embedding  \
0       0  one of the boys  [-0.6823418, 0.6486567, 0.3020579, 0.041714665...   
1       1  one of the boys  [-0.6823418, 0.6486567, 0.3020579, 0.041714665...   
2       2  one of the boys  [-0.6823418, 0.6486567, 0.3020579, 0.041714665...   
3       3  one of the boys  [-0.6823418, 0.6486567, 0.3020579, 0.041714665...   
4       4  one o

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [48]:
import os, sys
sys.path.append(os.path.dirname(os.path.join(os.path.dirname(os.getcwd()), 'neural-collaborative-filtering/src/')))

import torch
from tqdm import trange
from gmf import GMFEngine

In [17]:
trained_model_ckpt = '../neural-collaborative-filtering/src/checkpoints/gmf_factor8neg4-implict_Epoch47_HR1.0000_NDCG0.8431.model'

gmf_config = {'alias': 'gmf_factor8neg4-implict',
              'num_epoch': 200,
              'batch_size': 1024,
              # 'optimizer': 'sgd',
              # 'sgd_lr': 1e-3,
              # 'sgd_momentum': 0.9,
              # 'optimizer': 'rmsprop',
              # 'rmsprop_lr': 1e-3,
              # 'rmsprop_alpha': 0.99,
              # 'rmsprop_momentum': 0,
              'optimizer': 'adam',
              'adam_lr': 1e-3,
              'num_users': 7700,
              'num_items': 158,
              'bert_dim': 1024,
              'latent_dim': 8,
              'num_negative': 4,
              'l2_regularization': 0, # 0.01
              'use_cuda': False,
              'device_id': 0,
              'model_dir':'checkpoints/{}_Epoch{}_HR{:.4f}_NDCG{:.4f}.model'}

engine = GMFEngine(gmf_config)
state_dict = torch.load(trained_model_ckpt)
engine.model.load_state_dict(state_dict)

<All keys matched successfully>

In [45]:
batch_size = 1024
n_steps = len(cav_df)//batch_size

userIDs = torch.LongTensor(list(cav_df['userId']))
embeddings = torch.Tensor(list(cav_df['embedding']))

In [82]:
bottlenecks = []

engine.model.eval()
with torch.no_grad():
    for s in trange(0, n_steps):
        b = engine.model(userIDs[s*batch_size:(s+1)*batch_size],
                         embeddings[s*batch_size:(s+1)*batch_size])
        b = engine.model._get_bottleneck()
        bottlenecks.extend(list(b))
    # Last batch
    b = engine.model(userIDs[n_steps*batch_size:],
                         embeddings[n_steps*batch_size:])
    b = engine.model._get_bottleneck()
    bottlenecks.extend(list(b))

100%|██████████| 3383/3383 [00:07<00:00, 428.02it/s]


In [84]:
cav_df['bottleneck'] = bottlenecks

In [85]:
cav_df.head()

,userId,word,embedding,CAV_label,bottleneck
0,0,white woman,"[-0.680018, 0.57383853, -0.50681406, -0.333249...",0.0,[tensor(-0.2533)]
1,1,white woman,"[-0.680018, 0.57383853, -0.50681406, -0.333249...",0.0,[tensor(0.9172)]
2,2,white woman,"[-0.680018, 0.57383853, -0.50681406, -0.333249...",0.0,[tensor(-0.2931)]
3,3,white woman,"[-0.680018, 0.57383853, -0.50681406, -0.333249...",0.0,[tensor(0.5616)]
4,4,white woman,"[-0.680018, 0.57383853, -0.50681406, -0.333249...",0.0,[tensor(0.0050)]


In [86]:
cav_dat = 'cav.dat'
cav_df.to_pickle(cav_dat)